<a href="https://colab.research.google.com/github/ElioAlmiron/Starting_repository/blob/main/Opera%C3%A7%C3%B5es_na_Clear_Corretora.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**Extração de dados para declaração de imposto de renda de Notas de Corretagem - Clear Corretora**

Todo início de ano é necessário fazer a declaração de imposto de renda. E é uma das atividades mais chatas desse período - pior que pegar congestionamento na volta do Carvanal, que nos toma um bom tempo e nos gera certo stress. Sempre há o risco de fazer a declaração incorretamente e, como consequência, cair na malha fina da Receita Federal. E para quem opera investe em ações, há um ponto adicional: inserir as operações realizadas no ano fiscal. Para isso, é recomendado planilhar cada uma dessas transações para facilitar a declaração. 


Desse modo, montei um programa em Python que lê cada uma das Notas de Negociação e exporta cada uma das informações relevantes para declaração para um arquivo em Excel. Fiz isso tanto para ganhar tempo quanto para ter maior precisão nos dados, uma vez que a tarefa passa a ser automatizada, gerando segurança na informação passada à Receita Federal. 


Observação: utilizei como base a Nota de Corretagem da Clear do Corretora, do Grupo XP, mas o código pode ser adaptado para qualquer outra Nota de Negociação, uma vez que a lógica não é alterada.

**É importante ressaltar que não sou contador, mas sim engenheiro, e fiz esse material apenas para facilitar a minha vida e o disponibilizo a título de conhecimento. Caso desejem utilizá-lo, peço que não o considerem para as suas declarações sem antes validar cada um dos registros com todas as Notas de Negociação.**

##**Bibliotecas utilizadas:**



1.   Utilizei a biblioteca ***OS*** para interagir com o sistema operacional através do Python, acessando a pasta onde as Notas de Corretagem se encontram.

2.   Utilizei a biblioteca **PDF Plubber** para acessar os arquivos PDF pelo Python.

3.   Utilizei a biblioteca ***pandas*** para exportar o *output* para o Excel, que facilita caso haja interesse em enviar o arquivo a um contador.

##**Acessar as Notas de Negociação**

Primeiramente, é necessário salvar todas as Nota de Negociação em uma pasta, que será acessada pelo Python.


O comando ***os.chdir*** nos permite inserir o caminho onde essa pasta se encontra:

In [3]:
#importar OS
import os

In [4]:
os.chdir('/content/Notas de negociação')

Na sequência, é criada a lista ***Notas*** com cada um dos arquivos referentes às Notas de Negociação:

In [5]:
Notas = os.listdir()
Notas

['NotaNegociacao_418119_20200401.pdf',
 'NotaNegociacao_418119_20200319.pdf',
 'NotaNegociacao_418119_20200115.pdf',
 'NotaNegociacao_418119_20200610.pdf',
 'NotaNegociacao_418119_20201111.pdf',
 'NotaNegociacao_418119_20200415.pdf',
 'NotaNegociacao_418119_20200916.pdf',
 'NotaNegociacao_418119_20200218.pdf',
 'NotaNegociacao_418119_20200917.pdf',
 'NotaNegociacao_418119_20200212.pdf',
 'NotaNegociacao_418119_20200508.pdf',
 'NotaNegociacao_418119_20200318.pdf',
 'NotaNegociacao_418119_20201126.pdf',
 'NotaNegociacao_418119_20201222.pdf',
 'NotaNegociacao_418119_20200327.pdf',
 'NotaNegociacao_418119_20200325.pdf',
 'NotaNegociacao_418119_20200602.pdf',
 'NotaNegociacao_418119_20200309.pdf',
 'NotaNegociacao_418119_20200904.pdf',
 'NotaNegociacao_418119_20200420.pdf',
 'NotaNegociacao_418119_20200324.pdf',
 'NotaNegociacao_418119_20200430.pdf',
 'NotaNegociacao_418119_20200403.pdf',
 'NotaNegociacao_418119_20200525.pdf',
 'NotaNegociacao_418119_20200519.pdf',
 'NotaNegociacao_418119_2

E então a variável ***Quant_Notas*** salva a quantidade de Notas de Negociação. Isso facilita o acesso a cada nota, pois a leitura de todas será automatizada por meio de uma estrutura de repetição até que a última nota seja processada.

In [6]:
Quant_Notas = len(Notas)

##**Ler cada Nota de Negociação e salvar todos os registros em uma lista**

Com uma estrutura de repetição, cada Nota de Negociação será tratada, havendo o registro das seguintes informações:

*   Data da operação
*   Operação de Compra ou Venda
*   Tipo de Mercado
*   Empresa
*   Especificação da ação (ON, PN, Unit)
*   Quantidade
*   Preço da ação
*   Valor da Operação
*   Operação de Débito ou Crédito
*   Taxas e impostos















In [8]:
# Instalar o PDPF Plumber
!pip install pdfplumber -q

# importar o PDF Plumber
import pdfplumber

     |████████████████████████████████| 51kB 2.9MB/s 
     |████████████████████████████████| 5.6MB 6.7MB/s 
     |████████████████████████████████| 143kB 43.8MB/s 
     |████████████████████████████████| 1.9MB 46.1MB/s 


In [9]:
# lista que será utilizada para o registro de todas as operações

base_atual = []

# abrir o arquivo

# O while abaixo percorrerá todos os PDFs que estiverem no arquivo em questão
a = 0
while a < Quant_Notas:
  
  # abrir o PDF de índice "a"
  pdf = pdfplumber.open(Notas[a])
  
  # referenciar a primeira página do PDF (índice 0, onde estarão os dados)
  page = pdf.pages[0]
  
  # extrair informações da página
  text = page.extract_text()
  
  # quantidade de linhas dessa página
  quant_linhas = len(text.split('\n'))
  
  registros = [] # cada registro serão adicionados à lista 'registros'
  
  # leitura da data da operação, qu será única para cada nota de negociação

  Data = text.split('\n')[2][-10:]

  # cálculo das taxas em cada operação dessa Nota de Negociação
  
  taxas = 0

  x = -7 # x representa o primeiro algarismo antes da vírgula
  while text.split('\n')[-28][x : x + 1] != " ": # quando encontrar o vazio, não é mais parte do número
    x = x - 1 # se não encontrou o vazio, testa um campo para a esquerda
  y = text.split('\n')[-28][x + 1 : -2] # número
  
  taxas = taxas - float(y.replace('.','').replace(',','.'))
  
  
  x = -7 # x representa o primeiro algarismo antes da vírgula
  while text.split('\n')[-10][x : x + 1] != " ": # quando encontrar o vazio, não é mais parte do número
    x = x - 1 # se não encontrou o vazio, testa um campo para a esquerda
  y = text.split('\n')[-10][x + 1 : -2] # número
  
  taxas = abs(round(taxas + float(y.replace('.','').replace(',','.')), 2))


  # somatório de todos os valores de operações

  i = 19 # a próxima informação está na linha de índice 19, que é a linha onde está a primeira negociação

  soma_das_operacoes = 0

  # O while abaixo soma o valor de todas as operações, para que seja rateada a taxa da nota de corretagem inteira por todas operações
  
  while text.split('\n')[i][0] == "1":
    
    x = -7 # x representa o primeiro algarismo antes da vírgula
    while text.split('\n')[i][x : x + 1] != " ": # quando encontrar o vazio, não é mais parte do número
      x = x - 1 # se não encontrou o vazio, testa um campo para a esquerda
    y = text.split('\n')[i][x + 1 : -2] # número
    y = float(y.replace('.','').replace(',','.'))

    soma_das_operacoes = soma_das_operacoes + y
    
    i = i + 1


  i = 19 #  volta para a linha de índice 19 para buscar uma a uma todas as informações de cada negociação
  
  while i < quant_linhas:
  
    if text.split('\n')[i][0] == "1": # será avaliada cada linha com registro desde a primeira. Quando começar com algo diferente de "1", não será registro.
      # leitura da data da operação

      Data = text.split('\n')[2][-10:]

      # adição da data na lista de registros

      registros.append(Data)


      # C/V: retira o campo "C/V" e o adiciona na lista de registros
      CV = text.split('\n')[i][10].strip()
      registros.append(CV)


      # tipo_mercado: retira o campo "Tipo mercado" e o adiciona na lista de registros
      tipo_mercado = text.split('\n')[i][12:17]
      registros.append(tipo_mercado)


      # especificacao_do_titulo: retira o campo "Especificação do Título" e o adiciona na lista de registros
      
      j = 18
      espaco = 0
      
      while espaco < 2: # não pode chegar a dois espaços. Caso chegue, não é mais separação do nome (exemplo: "Santos Brasil" tem um só espaço)
        if text.split('\n')[i][j] == " ":
          espaco = espaco + 1
        j = j + 1
      

      nome = text.split('\n')[i][18:j-2]
      registros.append(nome)


      n = 0
      codigo = 0

      while n == 0: # "n" é o controle para sair do loop 
        if text.split('\n')[i][j] == " ":
          j = j + 1
        else:
          codigo = text.split('\n')[i][j:j+3]
          n = 1
      
      registros.append(codigo)

      # pela grande variação de quantidade de algarismos dos códigos, o próximo campo a ser buscado não será o primeiro após o código, mas sim o último da lista, passando a corrê-la do fim até esse ponto

      # DC: retira o campo "D/C" e o adiciona à lista de registros

      DC = text.split('\n')[i][-1]


      # valor: retira o campo "Valor Operação" e o adiciona à lista de registros

      x = -7 # x representa o primeiro algarismo antes da vírgula
      while text.split('\n')[i][x : x + 1] != " ": # quando encontrar o vazio, não é mais parte do número
        x = x - 1 # se não encontrou o vazio, testa um campo para a esquerda
      valor = text.split('\n')[i][x + 1 : -2] # número
     

      x = x - 4 # x - 4 é o último algarismo antes da vírgula no campo "Preço / Ajuste"
      y = x
      #print(text.split('\n')[i][a])
      # preco_ajuste: retira o campo "Preço / Ajuste" e o adiciona à lista de registros
      while text.split('\n')[i][x] != " ": #quando o valor procurado for vazio, terão sido encontrados todos os algarismos desse campo
        x = x - 1
        #print(text.split('\n')[i][x])
        y = y + 1
        #print(text.split('\n')[i][y])
      
      preco_ajuste = text.split('\n')[i][x + 1 : y + 3]


      x = x - 1
      y = x

      # quantidade: retira o campo "Quantidade" e o adiciona à lista de registros

      while text.split('\n')[i][x] != " ": #quando o valor procurado for vazio, terão sido encontrados todos os algarismos desse campo 
        x = x - 1 # caminha por todo o campo, até encontrar seu valor inteiro
        y = y + 1 # registra até qual algarismo deve ser considerado o valor da quantidade

      quantidade = text.split('\n')[i][x + 1: y - 2]

      # adiciona "quantidade", "preço ajuste", "valor" e "DC" à lista de registros

      registros.append(quantidade)
      registros.append(preco_ajuste)
      registros.append(valor)
      registros.append(DC)

      # taxas: rateia as taxas por cada uma das operações, a partir do valor
      
      taxas_2 = round(taxas * float(valor.replace('.','').replace(',','.')) / soma_das_operacoes,2)
      registros.append(taxas_2)


      # imprime o registro
      print('Registro', i- 18, 'arquivo' , a + 1 , ':', registros)
      base_atual.append(registros)

      registros = []
    i = i + 1

  a = a + 1



Registro 1 arquivo 1 : ['01/04/2020', 'C', 'VISTA', 'CENTAURO', 'ON ', '100', '21,74 ', '2.174,00', 'D', 0.67]
Registro 2 arquivo 1 : ['01/04/2020', 'V', 'VISTA', 'TAESA', 'PN ', '300', '8,21', '2.463,00', 'C', 0.76]
Registro 3 arquivo 1 : ['01/04/2020', 'C', 'VISTA', 'VIAVAREJO', 'ON ', '100', '4,66', '466,00', 'D', 0.14]
Registro 1 arquivo 2 : ['19/03/2020', 'V', 'VISTA', 'PETROBRAS', 'PN ', '500', '12,32 ', '6.160,00', 'C', 1.89]
Registro 1 arquivo 3 : ['15/01/2020', 'C', 'VISTA', 'SANTOS BR', 'ON ', '300', '8,36', '2.508,00', 'D', 0.77]
Registro 1 arquivo 4 : ['10/06/2020', 'V', 'VISTA', 'SID NACIONA', 'ON ', '100', '11,80 ', '1.180,00', 'C', 0.36]
Registro 2 arquivo 4 : ['10/06/2020', 'V', 'VISTA', 'SID NACIONA', 'ON ', '200', '11,80 ', '2.360,00', 'C', 0.72]
Registro 1 arquivo 5 : ['11/11/2020', 'C', 'VISTA', 'ISHARE SP50', 'CI ', '1', '207,86 2', '2.078,60', 'D', 0.64]
Registro 2 arquivo 5 : ['11/11/2020', 'C', 'VISTA', 'ISHARE SP50', 'CI ', '1', '207,86 2', '2.078,60', 'D', 0.6

A base se encontrará abaixo:

In [10]:
#print('Base atual:' , base_atual)
print('Base atual:' , base_atual)

Base atual: [['01/04/2020', 'C', 'VISTA', 'CENTAURO', 'ON ', '100', '21,74 ', '2.174,00', 'D', 0.67], ['01/04/2020', 'V', 'VISTA', 'TAESA', 'PN ', '300', '8,21', '2.463,00', 'C', 0.76], ['01/04/2020', 'C', 'VISTA', 'VIAVAREJO', 'ON ', '100', '4,66', '466,00', 'D', 0.14], ['19/03/2020', 'V', 'VISTA', 'PETROBRAS', 'PN ', '500', '12,32 ', '6.160,00', 'C', 1.89], ['15/01/2020', 'C', 'VISTA', 'SANTOS BR', 'ON ', '300', '8,36', '2.508,00', 'D', 0.77], ['10/06/2020', 'V', 'VISTA', 'SID NACIONA', 'ON ', '100', '11,80 ', '1.180,00', 'C', 0.36], ['10/06/2020', 'V', 'VISTA', 'SID NACIONA', 'ON ', '200', '11,80 ', '2.360,00', 'C', 0.72], ['11/11/2020', 'C', 'VISTA', 'ISHARE SP50', 'CI ', '1', '207,86 2', '2.078,60', 'D', 0.64], ['11/11/2020', 'C', 'VISTA', 'ISHARE SP50', 'CI ', '1', '207,86 2', '2.078,60', 'D', 0.64], ['11/11/2020', 'V', 'VISTA', 'JBS', 'ON ', '200', '21,15 ', '4.230,00', 'C', 1.3], ['15/04/2020', 'C', 'VISTA', 'IMC S/', 'ON ', '1.000 2', '2,73', '2.730,00', 'D', 0.83], ['16/09/20

##**Exportar a base para o Excel**

A base será exportada via pandas para o arquivo "Operações realizadas". Desse modo, o arquivo estará pronto para **validação** entre cada um de seus registros e as Notas de Corretagem.

In [11]:
import pandas as pd

df = pd.DataFrame(base_atual)
df.to_excel(excel_writer = "Operações realizadas.xlsx")